In [42]:
import pandas as pd
import fastText as ft
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.utils.np_utils import to_categorical
from sklearn.metrics import f1_score
from keras import regularizers
from keras import optimizers
from collections import Counter
from keras.activations import softmax
from keras.callbacks import EarlyStopping

In [2]:
model = ft.load_model("")

In [3]:
data = pd.read_csv("final-corpus.csv",sep=",")

In [4]:
data["first_200"] = data["text"].apply(lambda x: x.split(" ")[:200])

In [5]:
X = list(data["first_200"])


In [6]:
enc = LabelEncoder()
enc.fit(data["category"])
Y = enc.transform(data["category"])

In [7]:
X = [[model.get_word_vector(w) for w in doc] for doc in X]

In [8]:
X = pad_sequences(X, maxlen=200)

In [9]:
Y = to_categorical(Y)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, shuffle=True)

In [48]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')


model = Sequential()
model.add(Conv1D(kernel_size=10,filters = 50,input_shape=(200,300)))
model.add(Dropout(0.8))
model.add(Flatten())
model.add(Dense(300, activation="relu"))
model.add(Dropout(0.9))
model.add(Dense(30, activation="softmax"))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

model.fit(X_train, y_train,
          epochs=70,
          batch_size=400, shuffle=True, callbacks=[early_stop])
y_pred = model.predict(X_test, batch_size=100)

y_pred = [np.argmax(x) for x in y_pred]
y_true = [np.argmax(x) for x in y_test]

print(f1_score(y_true, y_pred, average="macro"))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_30 (Conv1D)           (None, 191, 50)           150050    
_________________________________________________________________
dropout_42 (Dropout)         (None, 191, 50)           0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 9550)              0         
_________________________________________________________________
dense_38 (Dense)             (None, 300)               2865300   
_________________________________________________________________
dropout_43 (Dropout)         (None, 300)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 30)                9030      
Total params: 3,024,380
Trainable params: 3,024,380
Non-trainable params: 0
_________________________________________________________________
